In [1]:
%cd ..

/home/ec2-user/uclresearchanalysis


In [2]:
import builtins
# builtins.uclresearch_topic = 'GIVENCHY' # 226984 entires
builtins.uclresearch_topic = 'HAWKING' # 4828104 entries
# builtins.uclresearch_topic = 'NYC'
# builtins.uclresearch_topic = 'FLORIDA'
from configuration import config
import math
from tqdm import tqdm
import multiprocessing
from multiprocessing import Pool
from IPython.display import display
import numpy as np
import pandas as pd
import time
import traceback

('Reading config file from location: '
 '/home/ec2-user/uclresearchanalysis/configuration/env.properties')
{'calculate': {'analysis': True,
               'network': True,
               'uniquetweets': True,
               'uniqueusers': True},
 'data': {'dates': ['2018-03-13', '2018-03-14', '2018-03-15'],
          'eventname': "hawking's death",
          'phrases': ['stephen%20hawking%20death',
                      'stephen%20hawking%20die',
                      'stephen%20hawking%20passed%20away'],
          'starttime': 'Mar 13 12:00:00 -0500 2018'},
 'path': {'crawl': {'followers': '/home/ec2-user/uclresearchanalysis/followers',
                    'friends': '/home/ec2-user/uclresearchanalysis/friends'},
          'cwd': '/home/ec2-user/uclresearchanalysis/data/hawking',
          'ml': '/home/ec2-user/uclresearchanalysis/data/hawking/pickle',
          'networkx': {'all': '/home/ec2-user/uclresearchanalysis/data/hawking/pickle/networkx_all.dat',
                       'frien

In [3]:
unique_users = config.load_users_dataframe()

Loading data file from path /home/ec2-user/uclresearchanalysis/data/hawking/pickle/users.dat
'Loaded 123533 entires'


In [4]:
unique_users = unique_users.sample(frac=1)
unique_users.head()

,text,time_lapsed,order,created_at,description,favourites_count,followers_count,followers_json,friends_count,friends_json,...,source_index,seed_index,generation,time_since_seed,user_created_days,normalized_statuses_count,normalized_followers_count,normalized_favourites_count,normalized_listed_count,normalized_friends_count
28948,RT @warrenleightTV: Stephen Hawking was born J...,259.15,28949.0,2011-04-10 18:28:13,For to be free is not merely to cast off ones ...,4173,222,[],506,"[425871040, 44196397, 872381003066748929, 1465...",...,13881,4,2,256.05,2529,2.915777,0.087782,1.650059,0.001582,0.200079
49608,RT @WIRED: From his birth on the anniversary o...,530.58,49609.0,2016-04-14 04:20:25,SFine Art Director. QOL Institute Director. IT...,2017,145,"[1681700538, 3000697008, 1262776610, 791401079...",486,"[63461143, 1720046887, 284188758, 403179950, 1...",...,5514,9,2,526.06,699,1.917024,0.207439,2.885551,0.000000,0.695279
80891,RT @h3h3productions: Learning of Stephen Hawki...,806.63,80892.0,2017-10-20 21:35:47,yo soy de una colombia diferente a la de Kika ...,28544,243,"[926147597378531328, 868436205121163264, 76749...",574,"[431892088, 985072906425196544, 4166593281, 16...",...,3454,333,2,793.9,144,46.611111,1.687500,198.222222,0.000000,3.986111
40642,RT @h3h3productions: Learning of Stephen Hawki...,437.30,40643.0,2016-03-19 21:42:45,LBSS,2667,46,"[85124669, 454296562, 961387023612436480, 8743...",74,"[798002577208201217, 961387023612436480, 18157...",...,6683,6683,1,384.37,724,0.392265,0.063536,3.683702,0.000000,0.102210
109379,RT @warrenleightTV: Stephen Hawking was born J...,1334.17,109380.0,2010-07-16 16:53:07,I’m always naked and hungry and tired,18533,809,"[2617506421, 844556722949832704, 1011427005785...",404,"[976939127860969472, 267856525, 96526617983625...",...,13881,4,2,1331.07,2798,18.557898,0.289135,6.623660,0.005718,0.144389


In [5]:
intervals = [30]
# Helper function for mean value calculation
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

In [6]:
unique_users.loc[0, 'followers_count'] + unique_users.loc[0, 'friends_count']

6315

In [7]:
in_degree = list(unique_users.friends_count)
out_degree = list(unique_users.followers_count)
degree = in_degree + out_degree

In [8]:
infectedNetworkStats = {}
for interval in intervals:
    statsDictionary = {}
    for timestamp in range(0, 24*60+1, interval):
        infected_dataframe = unique_users[unique_users.time_lapsed <= timestamp]
        statsDictionary[timestamp] = {}
        statsDictionary[timestamp]['totalNodeInfected'] = infected_dataframe.shape[0]
        statsDictionary[timestamp]['totalFollowers'] = sum(infected_dataframe.followers_count)
        statsDictionary[timestamp]['totalFriends'] = sum(infected_dataframe.friends_count)
    infectedNetworkStats[interval] = statsDictionary

In [9]:
def safe_division(x, y):
    if y == 0:
        return 0
    else:
        return x/y

In [10]:
def process_data(interval, start_index, end_index):
    features = {
        'label': [],
        # UsM: User metadata
        'UsM_deltaDays': [],
        'UsM_statusesCount': [],
        'UsM_followersCount': [],
        'UsM_favouritesCount': [],
        'UsM_friendsCount': [],
        'UsM_listedCount': [],
        'UsM_normalizedUserStatusesCount': [],
        'UsM_normalizedUserFollowersCount': [],
        'UsM_normalizedUserFavouritesCount': [],
        'UsM_normalizedUserListedCount': [],
        'UsM_normalizedUserFriendsCount': [],          
        'UsM_deltaDays0': [],
        'UsM_statusesCount0': [],
        'UsM_followersCount0': [],
        'UsM_favouritesCount0': [],
        'UsM_friendsCount0': [],
        'UsM_listedCount0': [],
        'UsM_normalizedUserStatusesCount0': [],
        'UsM_normalizedUserFollowersCount0': [],
        'UsM_normalizedUserFavouritesCount0': [],
        'UsM_normalizedUserListedCount0': [],
        'UsM_normalizedUserFriendsCount0': [],
        'UsM_deltaDays-1': [],
        'UsM_statusesCount-1': [],
        'UsM_followersCount-1': [],
        'UsM_favouritesCount-1': [],
        'UsM_friendsCount-1': [],
        'UsM_listedCount-1': [],
        'UsM_normalizedUserStatusesCount-1': [],
        'UsM_normalizedUserFollowersCount-1': [],
        'UsM_normalizedUserFavouritesCount-1': [],
        'UsM_normalizedUserListedCount-1': [],
        'UsM_normalizedUserFriendsCount-1': [],
        # TwM: Tweet metadata
        'TwM_t0': [],
        'TwM_tSeed0': [],
        'TwM_t-1': [],
        'TwM_tSeed-1': [],
        'TwM_tCurrent': [],
        # Nw: Network
        'Nw_degree': [],
        'Nw_inDegree': [],
        'Nw_outDegree': [],
        'Nw_degree0': [],
        'Nw_inDegree0': [],
        'Nw_outDegree0': [],
        'Nw_degree-1': [],
        'Nw_inDegree-1': [],
        'Nw_outDegree-1': [],
        'Nw_degreeSeed0': [],
        'Nw_inDegreeSeed0': [],
        'Nw_outDegreeSeed0': [],
        'Nw_degreeSeed-1': [],
        'Nw_inDegreeSeed-1': [],
        'Nw_outDegreeSeed-1': [],
        # SNw: Spreading Network
        'SNw_nFriendsInfected': [],
        'SNw_friendsInfectedRatio': [],
        'SNw_generation0': [],
        'SNw_generation-1': [],
        'SNw_timeSinceSeed0': [],
        'SNw_timeSinceSeed-1': [],
        'SNw_totalNodesInfected': [],
        'SNw_nodeInfectedCentrality': [],
        'SNw_totalInDegree': [],
        'SNw_totalOutDegree': [],
        'SNw_inDegreeCentrality': [],
        'SNw_inDegreeCentrality0': [],
        'SNw_inDegreeCentrality-1': [],
        'SNw_outDegreeCentrality': [],
        'SNw_outDegreeCentrality0': [],
        'SNw_outDegreeCentrality-1': [],
        'SNw_inDegreeCentralitySeed0':[],
        'SNw_outDegreeCentralitySeed0':[],
        'SNw_inDegreeCentralitySeed-1':[],
        'SNw_outDegreeCentralitySeed-1':[],
        # Stat: Statistical
        'Stat_average_kOut': [],
        'Stat_average_t': [],
        'Stat_average_deltaDays': [],
        'Stat_average_statusesCount': [],
        'Stat_average_followersCount': [],
        'Stat_average_favouritesCount': [],
        'Stat_average_friendsCount': [],
        'Stat_average_listedCount': [],
        'Stat_average_normalizedUserStatusesCount': [],
        'Stat_average_normalizedUserFollowersCount': [],
        'Stat_average_normalizedUserFavouritesCount': [],
        'Stat_average_normalizedUserListedCount': [],
        'Stat_average_normalizedUserFriendsCount': [],                
        'Stat_max_kOut': [],
        'Stat_min_kOut': []
    }

    with tqdm(total=len(list(unique_users[start_index: end_index].iterrows()))) as pbar: 
        for index, user_row in unique_users[start_index: end_index].iterrows():
            try:
                if len(user_row['source_candidates']) > 0 and ((user_row['source_index'] is not None) or math.isnan(user_row['time_lapsed'])):          
                    source_candidates = sorted(user_row['source_candidates'])
                    
                    # Assign the values here to save computation
                    first_source_index = source_candidates[0]
                    first_source_row = unique_users.loc[first_source_index]
                    first_source_seed_row = unique_users.loc[first_source_row['seed_index']]
                    
                    start_bar = int(first_source_row.time_lapsed / interval) + 1
                    # + 1 to get more infected data
                    end_bar = min((user_row['time_lapsed'] / interval) + 2, (24*60/interval)) 
                    bars = list(np.arange(start_bar * interval, end_bar * interval, interval))
                    number_of_bars = len(bars)
                    
                    
                    
                    for current_time in bars:
                        # all sources up to the current time
                        sources = [x for x in source_candidates if unique_users.loc[x].time_lapsed <= current_time]
                        sources_dataframe = unique_users.loc[sources]
                        degreeList = list(degree[i] for i in sources)
                        inDegreeList = list(in_degree[i] for i in sources)
                        outDegreeList = list(out_degree[i] for i in sources)
                        degreeList = list(unique_users.loc[i, 'followers_count'] + unique_users.loc[i, 'friends_count']  for i in sources)
                        timeList = [current_time - unique_users.loc[x].time_lapsed for x in sources]


                        
                        last_source_index = sources[-1]
                        last_source_row = unique_users.loc[last_source_index]
                        last_source_seed_row = unique_users.loc[last_source_row['seed_index']]
                        
                        #Extraction
                        if math.isnan(user_row['time_lapsed']):
                            features['label'].append(0)
                        else:
                            features['label'].append(int(current_time >= user_row['time_lapsed']))
                        # UsM: User metadata                    
                        features['UsM_deltaDays'].append(user_row['user_created_days'])
                        features['UsM_statusesCount'].append(user_row['statuses_count'])
                        features['UsM_followersCount'].append(user_row['followers_count'])
                        features['UsM_favouritesCount'].append(user_row['favourites_count'])
                        features['UsM_friendsCount'].append(user_row['friends_count'])
                        features['UsM_listedCount'].append(user_row['listed_count'])
                        features['UsM_normalizedUserStatusesCount'].append(user_row['normalized_statuses_count'])
                        features['UsM_normalizedUserFollowersCount'].append(user_row['normalized_followers_count'])
                        features['UsM_normalizedUserFavouritesCount'].append(user_row['normalized_favourites_count'])
                        features['UsM_normalizedUserListedCount'].append(user_row['normalized_listed_count'])
                        features['UsM_normalizedUserFriendsCount'].append(user_row['normalized_friends_count'])                 
                        features['UsM_deltaDays0'].append(first_source_row.user_created_days)
                        features['UsM_statusesCount0'].append(first_source_row.statuses_count)
                        features['UsM_followersCount0'].append(first_source_row.followers_count)
                        features['UsM_favouritesCount0'].append(first_source_row.favourites_count)
                        features['UsM_friendsCount0'].append(first_source_row.friends_count)
                        features['UsM_listedCount0'].append(first_source_row.listed_count)
                        features['UsM_normalizedUserStatusesCount0'].append(first_source_row.normalized_statuses_count)
                        features['UsM_normalizedUserFollowersCount0'].append(first_source_row.normalized_followers_count)
                        features['UsM_normalizedUserFavouritesCount0'].append(first_source_row.normalized_favourites_count)
                        features['UsM_normalizedUserListedCount0'].append(first_source_row.normalized_listed_count)
                        features['UsM_normalizedUserFriendsCount0'].append(first_source_row.normalized_friends_count)
                        features['UsM_deltaDays-1'].append(last_source_row.user_created_days)
                        features['UsM_statusesCount-1'].append(last_source_row.statuses_count)
                        features['UsM_followersCount-1'].append(last_source_row.followers_count)
                        features['UsM_favouritesCount-1'].append(last_source_row.favourites_count)
                        features['UsM_friendsCount-1'].append(last_source_row.friends_count)
                        features['UsM_listedCount-1'].append(last_source_row.listed_count)
                        features['UsM_normalizedUserStatusesCount-1'].append(last_source_row.normalized_statuses_count)
                        features['UsM_normalizedUserFollowersCount-1'].append(last_source_row.normalized_followers_count)
                        features['UsM_normalizedUserFavouritesCount-1'].append(last_source_row.normalized_favourites_count)
                        features['UsM_normalizedUserListedCount-1'].append(last_source_row.normalized_listed_count)
                        features['UsM_normalizedUserFriendsCount-1'].append(last_source_row.normalized_friends_count) 
                        # TwM: Tweet metadata
                        features['TwM_t0'].append(round(timeList[0], 1))
                        features['TwM_tSeed0'].append(round(current_time - first_source_seed_row['time_lapsed'], 1))
                        features['TwM_t-1'].append(round(timeList[-1], 1))
                        features['TwM_tSeed-1'].append(round(current_time - last_source_seed_row['time_lapsed'], 1))
                        features['TwM_tCurrent'].append(current_time)
                        # Nw: Network
                        features['Nw_degree'].append(degree[index])
                        features['Nw_inDegree'].append(in_degree[index])
                        features['Nw_outDegree'].append(out_degree[index])
                        features['Nw_degree0'].append(degree[first_source_index])
                        features['Nw_inDegree0'].append(in_degree[first_source_index])
                        features['Nw_outDegree0'].append(out_degree[first_source_index])
                        features['Nw_degree-1'].append(degree[last_source_index])
                        features['Nw_inDegree-1'].append(in_degree[last_source_index])
                        features['Nw_outDegree-1'].append(out_degree[last_source_index])
                        features['Nw_degreeSeed0'].append(degree[first_source_row['seed_index']])
                        features['Nw_inDegreeSeed0'].append(in_degree[first_source_row['seed_index']])
                        features['Nw_outDegreeSeed0'].append(out_degree[first_source_row['seed_index']])
                        features['Nw_degreeSeed-1'].append(degree[last_source_row['seed_index']])
                        features['Nw_inDegreeSeed-1'].append(in_degree[last_source_row['seed_index']])
                        features['Nw_outDegreeSeed-1'].append(out_degree[last_source_row['seed_index']])
                        # SNw: Spreading Network
                        features['SNw_nFriendsInfected'].append(len(sources))
                        features['SNw_friendsInfectedRatio'].append(safe_division(len(sources), user_row['friends_count']))
                        features['SNw_generation0'].append(first_source_row['generation'])
                        features['SNw_generation-1'].append(last_source_row['generation'])
                        features['SNw_timeSinceSeed0'].append(first_source_row['time_since_seed'])
                        features['SNw_timeSinceSeed-1'].append(last_source_row['time_since_seed'])
                        features['SNw_totalNodesInfected'].append(infectedNetworkStats[interval][current_time]['totalNodeInfected'])
                        features['SNw_nodeInfectedCentrality'].append(len(sources)/infectedNetworkStats[interval][current_time]['totalNodeInfected'])
                        features['SNw_totalInDegree'].append(infectedNetworkStats[interval][current_time]['totalFriends'])
                        features['SNw_totalOutDegree'].append(infectedNetworkStats[interval][current_time]['totalFollowers'])
                        features['SNw_inDegreeCentrality'].append(in_degree[index]/infectedNetworkStats[interval][current_time]['totalFriends'])
                        features['SNw_inDegreeCentrality0'].append(in_degree[first_source_index]/infectedNetworkStats[interval][current_time]['totalFriends'])
                        features['SNw_inDegreeCentrality-1'].append(in_degree[last_source_index]/infectedNetworkStats[interval][current_time]['totalFriends'])
                        features['SNw_outDegreeCentrality'].append(out_degree[index]/infectedNetworkStats[interval][current_time]['totalFollowers'])
                        features['SNw_outDegreeCentrality0'].append(out_degree[first_source_index]/infectedNetworkStats[interval][current_time]['totalFollowers'])
                        features['SNw_outDegreeCentrality-1'].append(out_degree[last_source_index]/infectedNetworkStats[interval][current_time]['totalFollowers'])
                        features['SNw_inDegreeCentralitySeed0'].append(in_degree[first_source_row['seed_index']]/infectedNetworkStats[interval][current_time]['totalFriends'])
                        features['SNw_outDegreeCentralitySeed0'].append(out_degree[first_source_row['seed_index']]/infectedNetworkStats[interval][current_time]['totalFollowers'])
                        features['SNw_inDegreeCentralitySeed-1'].append(in_degree[last_source_row['seed_index']]/infectedNetworkStats[interval][current_time]['totalFriends'])
                        features['SNw_outDegreeCentralitySeed-1'].append(out_degree[last_source_row['seed_index']]/infectedNetworkStats[interval][current_time]['totalFollowers'])
                        # Stat: Statistical
                        features['Stat_average_kOut'].append(round(mean(degreeList), 1))
                        features['Stat_average_t'].append(round(mean(timeList), 1))
                        features['Stat_average_deltaDays'].append(sources_dataframe.user_created_days.mean())
                        features['Stat_average_statusesCount'].append(sources_dataframe.statuses_count.mean())
                        features['Stat_average_followersCount'].append(sources_dataframe.followers_count.mean())
                        features['Stat_average_favouritesCount'].append(sources_dataframe.favourites_count.mean())
                        features['Stat_average_friendsCount'].append(sources_dataframe.friends_count.mean())
                        features['Stat_average_listedCount'].append(sources_dataframe.listed_count.mean())
                        features['Stat_average_normalizedUserStatusesCount'].append(sources_dataframe.normalized_statuses_count.mean())
                        features['Stat_average_normalizedUserFollowersCount'].append(sources_dataframe.normalized_followers_count.mean())
                        features['Stat_average_normalizedUserFavouritesCount'].append(sources_dataframe.normalized_favourites_count.mean())
                        features['Stat_average_normalizedUserListedCount'].append(sources_dataframe.normalized_listed_count.mean())
                        features['Stat_average_normalizedUserFriendsCount'].append(sources_dataframe.normalized_friends_count.mean())
                        features['Stat_max_kOut'].append(max(degreeList))
                        features['Stat_min_kOut'].append(min(degreeList))
                    processed_dataframe = pd.DataFrame(features)
            except Exception as ex:
                print(ex)
                traceback.print_exc()
            pbar.update(1)
    return processed_dataframe

In [11]:
number_of_processes = multiprocessing.cpu_count()
print('Will start {} processes'.format(number_of_processes))
for interval in intervals:
    with Pool(number_of_processes) as pool:
        parameters = []
        number_of_users = len(unique_users.index)
        task_size = math.ceil(number_of_users/number_of_processes)
        for i in range(number_of_processes):
            start_index = i * task_size
            end_index = min((i + 1) * task_size, number_of_users)
            parameters.append((interval, start_index, end_index))
        dataframe_results = pool.starmap(process_data, parameters)
        
    result = pd.DataFrame()
    result = result.append(dataframe_results)
    config.dump_ml_data(result, interval)
    print('extracted {} of rows'.format(len(result.index)))
    display(result)

Will start 16 processes


 21%|██        | 1633/7721 [33:31<3:54:13,  2.31s/it] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 47%|████▋     | 3637/7721 [2:37:50<5:49:44,  5.14s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 53%|█████▎    | 4076/7721 [3:13:45<5:48:35,  5.74s/it] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--Noteb

Dumping data to path /home/ec2-user/uclresearchanalysis/data/hawking/pickle/30_data.dat
Finished dumping data to path /home/ec2-user/uclresearchanalysis/data/hawking/pickle/30_data.dat
extracted 2033529 of rows


,label,UsM_deltaDays,UsM_statusesCount,UsM_followersCount,UsM_favouritesCount,UsM_friendsCount,UsM_listedCount,UsM_normalizedUserStatusesCount,UsM_normalizedUserFollowersCount,UsM_normalizedUserFavouritesCount,...,Stat_average_favouritesCount,Stat_average_friendsCount,Stat_average_listedCount,Stat_average_normalizedUserStatusesCount,Stat_average_normalizedUserFollowersCount,Stat_average_normalizedUserFavouritesCount,Stat_average_normalizedUserListedCount,Stat_average_normalizedUserFriendsCount,Stat_max_kOut,Stat_min_kOut
0,0,2529,7374,222,4173,506,4,2.915777,0.087782,1.650059,...,26650.000000,2435.000000,493.000000,18.812332,13.985829,10.206817,0.188817,0.932593,38952,38952
1,0,2529,7374,222,4173,506,4,2.915777,0.087782,1.650059,...,18708.500000,1963.500000,464.000000,17.330852,25.961708,6.980501,0.170245,0.726408,110297,38952
2,0,2529,7374,222,4173,506,4,2.915777,0.087782,1.650059,...,18708.500000,1963.500000,464.000000,17.330852,25.961708,6.980501,0.170245,0.726408,110297,38952
3,0,2529,7374,222,4173,506,4,2.915777,0.087782,1.650059,...,18708.500000,1963.500000,464.000000,17.330852,25.961708,6.980501,0.170245,0.726408,110297,38952
4,0,2529,7374,222,4173,506,4,2.915777,0.087782,1.650059,...,18708.500000,1963.500000,464.000000,17.330852,25.961708,6.980501,0.170245,0.726408,110297,38952
5,1,2529,7374,222,4173,506,4,2.915777,0.087782,1.650059,...,12996.333333,1397.000000,335.666667,13.730798,18.739659,4.818603,0.121785,0.511971,110297,13911
6,1,2529,7374,222,4173,506,4,2.915777,0.087782,1.650059,...,12996.333333,1397.000000,335.666667,13.730798,18.739659,4.818603,0.121785,0.511971,110297,13911
7,0,699,1340,145,2017,486,0,1.917024,0.207439,2.885551,...,270.000000,430.000000,47684.000000,46.892127,1814.176520,0.075651,13.360605,0.120482,6475226,6475226
8,0,699,1340,145,2017,486,0,1.917024,0.207439,2.885551,...,270.000000,430.000000,47684.000000,46.892127,1814.176520,0.075651,13.360605,0.120482,6475226,6475226
9,0,699,1340,145,2017,486,0,1.917024,0.207439,2.885551,...,270.000000,430.000000,47684.000000,46.892127,1814.176520,0.075651,13.360605,0.120482,6475226,6475226
